Устанавливаем Transformers и Datasets

In [1]:
! pip install datasets transformers evaluate

In [2]:
import numpy as np
from datasets import load_dataset
import evaluate
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertForQuestionAnswering
from transformers import AutoTokenizer
from transformers import default_data_collator
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from tqdm.auto import tqdm

Импортируем Transformers

In [3]:
import transformers

Загружаем Датасет с HuggingFace, Задав предварительно некоторые параметры. Загружаем Stanford Question Answering Dataset
[squad](https://rajpurkar.github.io/SQuAD-explorer/).
C помощью функций load_dataset и load_matric мы сможем загрузить данные и получить метрики которые нам нужно сипользовать для оценки

In [4]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

from datasets import load_dataset, load_metric
datasets = load_dataset("squad")

Для ускорения обучения обрежем датасет немного.

In [5]:
datasets['train'] = datasets['train'].select( range(5000))
datasets['validation'] = datasets['validation'].select(range(500))

объект `datasets` представляет собой [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), который состоит из одного ключа для набора данных для обучения, проверки и тетсирования

In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 500
    })
})

Конкретный элемент можно получить следующим образом

In [7]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


Здесь мы видим, что ответы обозначены их начальной позицией в тексте (здесь, символ 515) и их полным текстом, который является подстрокой контекста.

## Предварительная обработка обучающих данных

Прежде чем мы сможем передать эти тексты в нашу модель, нам необходимо их предварительно обработать. Сделаем это с помощью Transformers `Tokenizer` который токенезирует вхожные данные

Для этого создаем экземпляр нашего токенизатора с помощью метода  `AutoTokenizer.from_pretrained` который гарантирует:

- получаем токенизатор, соответствующий архитектуре модели, которую хотим использовать,
- загружаем словарь, используемый при предварительной тренировке этой конкретной контрольной точки.

Этот словарь будет кэширован, поэтому он не будет загружаться снова при следующем запуске ячейки.

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)



При работе с очень длиными текстами в "обычных" случаях можно просто урезать текст до определенной длины, однако в данном случае это может привезти к трезанию части где у нас хранится ответ на вопрос. Чтобы справиться с этим, позволим одному (длинному) примеру в нашем наборе данных предоставлять несколько входных объектов, длина каждого из которых короче максимальной длины модели (или той, которую мы установили в качестве гиперпараметра). Кроме того, на тот случай, если ответ лежит в том месте, где мы разделяем длинный контекст, мы допускаем некоторое перекрытие между генерируемыми нами функциями, управляемыми гиперпараметром `doc_stride`:

In [9]:
max_length = 384 # Максимальная длина (question и context)
doc_stride = 128 # Размер перекрытия между двумя частями при его разделении

Найдем более длиный текст чем заданная максимальная длина и посмотрим для начала без усечения:

In [10]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

len(tokenizer(example["question"], example["context"])["input_ids"])

396

Если мы сделаем усечение, мы потенциально можем потерять информацию с ответом

In [11]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

Мы никогда не хотим усекать вопрос, а только context, иначе будет выбрано truncation=only_second. Теперь токенизатор может возврщать спискок усеченной max_length, для этого нужно сообщить токенизатору с помощью return_overflowing_tokens=True и передав stride:

In [12]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

теперь мы имеем не один список, а два:

In [13]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

И можно увидеть перекрытие:

In [14]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

Для определения в какой из двух частей находится ответ и сопоставить токены в исходном тексте. Для этого мы ставим return_offset_mapping=True:

In [15]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

Таким образом для каждого входящего input_ids, указывается начальный и конечный симовл токена. Самый первый токен это ([CLS]) и он (0, 0) и второй токен соответвтсвует символам с  0 до 3 например вопроса:

In [16]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


Таким образом, используя это сопоставление, можно найти  положение начальных и конечных токенов нашего ответа в заданном объекте. Нам просто нужно различать, какие части offsets соответствуют вопросу, а какие — контексту, именно здесь используем метод sequence_ids нашего tokenized_example:

In [17]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Метод возвращает `None` для специальных токенов, затем 0 или 1 в зависимости от того, происходит ли соответствующий токен из первого предложения (вопрос) или второго (контекст). Теперь, имея все это, мы можем найти первый и последний токен ответа в одном из наших входных объектов (или, если ответа нет в этом объекте):


In [18]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Начальный индекс токена текущего диапазона в тексте
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# Индекс конечного токена текущего диапазона в тексте.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# определяем выходит ли ответ за пределы диапазона (в этом случае эта функция помечается индексом CLS).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # перемещаем token_start_index и token_end_index в конец ответа.
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

23 26


Можем проверить исравнить ответы:

In [19]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


Чтобы этот блокнот работал с любыми моделями, нам необходимо учитывать особый случай, когда модель ожидает заполнения слева (в этом случае мы меняем порядок вопросов и контекст):

In [20]:
pad_on_right = tokenizer.padding_side == "right"

Теперь объединим все в одну функцию, которую мы применим к нашему обучающему набору. В случае невозможных ответов (ответ находится в другой функции, заданной в примере с длинным контекстом), мы устанавливаем индекс cls как для начальной, так и для конечной позиции. Так же можно исключить эти примеры установив флаг `allow_impossible_answers = False`.

In [21]:
def prepare_train_features(examples):
    # Слева от некоторых вопросов много пробелов, которые не несут информации и затрудняют усечение
    # которое займет много места, поэтому удаляем лишние пробелы слева
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Поскольку один сонтекст может иметь большею длину и может дать несколько текстов
    # этот ключ даст нам это понимание
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # offset mappings даст нам карту от токена до позиции символа в исходном тектсе.
    # это поможет нам вычилить позиции начальные и конечные
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Отметим эти позиции
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # Если нет ответа будем помечать ответы индексом CLS.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # последовательность на примере, что бы понимать где вопрос, а где ответ.
        sequence_ids = tokenized_examples.sequence_ids(i)

        # В одном примере может быть несколько интервалов, здесь индекс интервала, который содержит нужный текст
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # Если ответа нет устанавливаем индекс CLS в качестве ответа.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # индекс начального и конечного символа ответата в тексте.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # начальный индекс токена текущего диапазона в тексте.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # конечный индекс токена текущего диапазона в тексте.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Определяем выходит ли ответ за пределы диапазона - тогда помечаем индексом CLS.
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # иначе перемещаем token_start_index и token_end_index в два конца ответа.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

Эта функция работает с одним или несколькими примерами. В случае нескольких примеров токенизатор вернет список списков для каждого ключа:

In [22]:
features = prepare_train_features(datasets['train'][:5])
features

{'input_ids': [[101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 1017, 11342, 1998, 1996, 2

Чтобы применить эту функцию ко всем предложениям (или парам предложений) в нашем наборе данных, мы просто используем метод `map` для нашего `dataset` объекта.
Это применит функцию ко всем элементам всех разбиений в наборе `dataset`, и наши данные обучения, проверки и тестирования будут предварительно обработаны с помощью одной команды. Поскольку наша предварительная обработка меняет количество выборок, нам необходимо удалить старые столбцы при ее применении.

In [23]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Более того, результаты автоматически кэшируются библиотекой наборов данных, чтобы не тратить время на этот шаг при следующем запуске блокнота. Библиотека наборов данных обычно достаточно умна, чтобы обнаружить, когда функция, которую вы передаете на карту, изменилась (и, следовательно, требует не использовать данные кэша). Например, он правильно определит, измените ли вы задачу в первой ячейке и перезапустите блокнот. Datasets предупреждает вас, когда он использует кешированные файлы, можно передать `load_from_cache_file=False` в функции map, чтобы не использовать кешированные файлы и принудительно применить предварительную обработку снова.
`patched=True` сделано для того, чтобы в полной мере использовать преимущества быстрого токенизатора, который загружен ранее, который будет использовать многопоточность для одновременной обработки текстов в пакете.

## ТОчная настройка модели

Теперь данные подготовленны и можно загружать предобученную модель и начтроить её. Поскольку задача - ответы н авопросы используем класс `AutoModelForQuestionAnswering`. Как и в случае с токенизатором, `from_pretrained` метод загрузит и кэширует для нас модель:

In [24]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model = DistilBertForQuestionAnswering.from_pretrained(model_checkpoint).to(device)
# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

cuda


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Чтобы создать экземпляр `Trainer`, нам нужно будет определить еще три вещи. Самым важным из них является TrainingArguments — класс, содержащий все атрибуты для настройки обучения. Обязательно требуется имя модели, которое будет использоваться для сохранения контрольных точек модели, а все остальные аргументы являются необязательными:

In [25]:
# import accelerate

# accelerate.__version__

In [26]:
!pip install transformers[torch]
!pip install accelerate -U

import accelerate

accelerate.__version__

'0.24.1'

Разбиваем датасеты на тренировочную и валидационную

In [27]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir='./fine_tuned_model',
    evaluation_strategy = "no",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=100,
    logging_steps=200
)

Мы оценим нашу модель и вычислим метрики в следующем разделе (это очень долгая операция, поэтому мы будем вычислять потери оценки только во время обучения).

Затем нам просто нужно передать все это вместе с нашими наборами данных в `Trainer`:

In [28]:
data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Теперь мы можем точно настроить нашу модель, просто вызвав метод `train`:

In [29]:
trainer.train()

Step,Training Loss
200,3.818800
400,2.426500
600,1.946100
800,1.637900


TrainOutput(global_step=957, training_loss=2.3057640498210143, metrics={'train_runtime': 592.2804, 'train_samples_per_second': 25.802, 'train_steps_per_second': 1.616, 'total_flos': 1497480506016768.0, 'train_loss': 2.3057640498210143, 'epoch': 3.0})

Сохраняем модель на всякий случай, что б нам в случае перезагрузки не приходилось переучивать модель

In [30]:
trainer.save_model("test-squad-trained")

## Оценка

Оценка нашей модели потребует немного больше работы, поскольку нам нужно будет сопоставить прогнозы нашей модели с частями контекста. Сама модель предсказывает логиты для начала и окончания наших ответов: если мы возьмем пакет из нашего проверочного dataloader, вот результат, который дает нам наша модель.

In [31]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])


Выходные данные модели — это объект, похожий на dict, который содержит потери (поскольку мы предоставили метки), начальные и конечные логиты. Для наших прогнозов потери нам не понадобятся, давайте посмотрим на логиты:

In [32]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([64, 384]), torch.Size([64, 384]))

У нас есть один логит для каждой функции и каждого токена. Самый очевидный способ предсказать ответ для каждого навыка — это принять индекс максимума начальных логитов в качестве начальной позиции, а индекс максимума конечных логитов в качестве конечной позиции.

In [33]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([150,  46, 147, 147, 113,  16,  72,  42, 162,  41,  73, 145,  80, 149,
         170,  35,  62, 163, 163, 163,  77, 160,  42,  53, 159,  46, 146,  71,
         162,  44,  27, 133,  66,  40,  87,  44,  43,  41, 122,  25,   0,  33,
          53, 122,  95,  25,  43, 132,  42,  29,  44,  46,  24,  44,  65,  58,
          81,  20,  68,  72,  25,  36,  55,  43], device='cuda:0'),
 tensor([ 47,  47, 149, 149, 114, 150,  75,  43, 150, 147,  76, 147,  83, 151,
         158, 152,  63, 151, 151, 151,  80,  74,  43,  43, 147, 152, 148,  74,
         150,  27,  28, 133,  66,  44,  89,  45,  47,  85, 123,  27,   0,  34,
           0, 123,  97,  26,  44, 132,  43,  30,  45,  50,  25,  48,  65,  59,
          81,  22,  60,  72,  25,  36,  58,  43], device='cuda:0'))

Во многих случаях это будет отлично работать, но что, если этот прогноз даст нам нечто невозможное: начальная позиция может быть больше конечной позиции или указывать на фрагмент текста в вопросе, а не на ответ. В этом случае мы, возможно, захотим посмотреть на второй лучший прогноз, чтобы увидеть, дает ли он возможный ответ, и вместо этого выбрать его.

Однако выбрать второй лучший ответ не так просто, как выбрать лучший: является ли это вторым лучшим индексом в начальных логитах с лучшим индексом в конечных логитах? Или лучший индекс в начальных логитах со вторым лучшим индексом в конечных логитах? А если и второй лучший ответ невозможен, то третий лучший ответ становится еще сложнее.

Для классификации наших ответов мы будем использовать оценку, полученную путем сложения начального и конечного логитов. Мы не будем пытаться упорядочить все возможные ответы и ограничимся гиперпараметром, который мы называем `n_best_size`. Мы выберем лучшие индексы в начальной и конечной логитах и ​​соберем все предсказываемые ответы. Проверив, действителен ли каждый из них, мы отсортируем их по баллам и оставим лучший. Вот как мы могли бы сделать это с первой функцией в пакете:

In [39]:
n_best_size = 20

import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# СОбираем индексы лучших начальных/конечных логитов:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # проверяем что ответ находится внутри контекста
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # возвращаем исходную подстроку соответствующую ответу в контексте
                }
            )

А затем мы сможем отсортировать действительные_ответы по их `score` и оставить только лучший. Остается только один вопрос: как проверить, находится ли данный диапазон внутри контекста (а не вопроса), и как вернуть текст внутри. Для этого нам нужно добавить две вещи к нашим функциям проверки:
- идентификатор примера, сгенерировавшего функцию (поскольку каждый пример может генерировать несколько функций, как было показано ранее);
- отображение смещения, которое даст нам карту индексов токенов с позициями символов в контексте..

Вот почему мы повторно обработаем набор проверки с помощью следующей функции, немного отличающейся от `prepare_train_features`:

In [40]:
def prepare_validation_features(examples):

    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

И, как и раньше, мы можем легко применить эту функцию к нашему набору проверки:

In [41]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)


Теперь мы можем получить прогнозы для всех функций, используя метод `Trainer.predict`:

In [42]:
raw_predictions = trainer.predict(validation_features)

`Trainer` скрывает столбцы, которые не используются моделью (здесь example_id и offset_mapping, которые нам понадобятся для постобработки), поэтому мы устанавливаем их обратно:

In [43]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))


Теперь мы можем усовершенствовать тест, который у нас был раньше: поскольку мы установили `None` в сопоставлениях смещений, когда оно соответствует части вопроса, легко проверить, находится ли ответ полностью внутри контекста. Мы также исключаем очень длинные ответы из наших соображений (с помощью гиперпараметра, который мы можем настроить)

In [44]:
max_answer_length = 30

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 4.5308247, 'text': 'Super Bowl L'},
 {'score': 4.5147057, 'text': 'Denver Broncos'},
 {'score': 3.445607,
  'text': 'American Football Conference (AFC) champion Denver Broncos'},
 {'score': 2.0781221,
  'text': 'National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 1.9335346,
  'text': 'Super Bowl 50 was an American football game to determine the champion of the National Football League'},
 {'score': 1.7985426,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10'},
 {'score': 1.692863, 'text': 'L'},
 {'score': 1.6376653,
  'text': 'American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 1.5238854,
  'text': 'Roman numerals (under which the game would have been known as "Super Bowl L'},
 {'score': 1.4937072,
  'text': 'Su

Сравним с реальным ответом

In [45]:
datasets["validation"][0]["answers"]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

Моедль выбрала почти правильный ответ)

С первым предсказанием относительно хорошо справилаись.
Для других нам понадобится карта между примерами и соответствующими им предсказаниям. Кроме того, поскольку один пример может давать несколько предложений, нам нужно будет собрать все ответы по всем предсказаниям, сгенерированным данным примером, а затем выбрать лучший. Следующий код создает карту из примера индекса с соответствующими индексами предсказаний:

In [47]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

Затем мы прогнозируем невозможный ответ, если score превышает score лучшего не невозможного ответа. Все вместе это дает нам функцию постобработки:
Затем мы прогнозируем невозможный ответ, если этот балл превышает балл лучшего не невозможного ответа. Все вместе это дает нам функцию постобработки:

In [53]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}


        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions


И мы можем применить нашу функцию постобработки к нашим необработанным прогнозам:

In [54]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 500 example predictions split into 520 features.


  0%|          | 0/500 [00:00<?, ?it/s]

In [55]:
metric = load_metric("squad")

<ipython-input-55-7143d13414c2>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")


In [56]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 50.6, 'f1': 56.427215886968206}